In [14]:
!pip install selenium

You should consider upgrading via the 'c:\users\ekkic\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait   # 해당 태그를 기다림
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException    # 태그가 없는 예외 처리
from selenium.webdriver.common.keys import Keys
from urllib.parse import urlparse
import time
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
driver = webdriver.Chrome('./chromedriver.exe')

url = 'https://www.transfermarkt.com/wettbewerbe/europa'
driver.get(url)

injuryList = []
gameMissedList = []
seasonList = []
fromList = []
toList = []
dayMissedList = []
playerAddList = []

leagueList = [0,0,0,0,0,0,0]

for l in range(0,7):
    #league에 맞는 xpath를 넣은건데, 깔끔하게 자동화하려다가 실패함
    inXpath = ''
    clubNum = 0
    if l == 0:
        inXpath = '//*[@id="yw1"]/table/tbody/tr[2]/td[1]/table/tbody/tr/td[2]/a'
        clubNum = 20
    elif l == 1:
        inXpath ='//*[@id="yw1"]/table/tbody/tr[3]/td[1]/table/tbody/tr/td[2]/a'
        clubNum = 20
    elif l == 2:
        inXpath ='//*[@id="yw1"]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[2]/a'
        clubNum = 20
    elif l == 3:
        inXpath ='//*[@id="yw1"]/table/tbody/tr[5]/td[1]/table/tbody/tr/td[2]/a'
        clubNum = 18
    elif l == 4:
        inXpath ='//*[@id="yw1"]/table/tbody/tr[6]/td[1]/table/tbody/tr/td[2]/a'
        clubNum = 20
    elif l == 5:
        inXpath ='//*[@id="yw1"]/table/tbody/tr[7]/td[1]/table/tbody/tr/td[2]/a'
        clubNum = 18
    elif l == 6:
        inXpath ='//*[@id="yw1"]/table/tbody/tr[8]/td[1]/table/tbody/tr/td[2]/a'
        clubNum = 18
    element = driver.find_element_by_xpath(inXpath)
    driver.execute_script("arguments[0].click();", element)
    leagueUrl = driver.current_url
    
    for k in range(0,clubNum):
        element = driver.find_elements_by_class_name("vereinprofil_tooltip")[k*3]
        driver.execute_script("arguments[0].click();", element)
        init_url = driver.current_url
        test = driver.find_elements(By.CLASS_NAME, 'tooltipstered')
        testList = list();
        for i in test:
            if i.text == '':
                continue
            testList.append(i.text)
        playerList = []
        injuryCntList = [] #선수당 부상기록의 개수
    
        #한팀에 선수가 몇명있는가?
        playerNum = 0
        squad = driver.find_elements(By.CLASS_NAME, 'rn_nummer')
        squadList = list();
        for i in squad:
            if i.text == '':
                continue
            squadList.append(i.text)
            playerNum = playerNum+1
    
        print("playerNum")
        print(playerNum)
    
        for i in range(0,playerNum):
            playerList.append(testList[i])
        for i in range(0,playerNum):
            curPlayer = playerList[i]
            print("Player")
            print(curPlayer)
            id = i*2
            element = driver.find_elements_by_class_name("tooltipstered")[id]
            driver.execute_script("arguments[0].click();", element)
            curUrl = driver.current_url
            nextUrl = curUrl.replace('profil','verletzungen')
            driver.get(nextUrl)
    
            page = nextUrl
            pageTree = requests.get(page, headers=headers)
            pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
            Injuries = pageSoup.find_all("td", {"class": "hauptlink"})
            #players[even] = injury name, players[odd] = missed game
            Others = pageSoup.find_all("td", {"class": "zentriert"})
            #Others[3n] = season, Others[3n+1] = from, Other[3n+2] = to
            Days = pageSoup.find_all("td", {"class": 'rechts'})
            #Days[even] = day missed
        
            #Latest Rumors Table에 hauptlink와 zentriert, rechts가 존재함, 이걸 제거
            cnt = 0
            test2 = driver.find_elements(By.CLASS_NAME, 'tooltipstered')
            testList2 = list();
            for i in test2:
                if i.text == '':
                    continue
                testList2.append(i.text)
                cnt = cnt+1
            cnt = cnt - 2;
            #print("Injuries legnth")
            #print(len(Injuries) - cnt)
        
            #print("Others length")
            #print(len(Others) - cnt)
            
            #print("Days length")
            #print(len(Days) - cnt)
            #print("j")
        
            totalLength = (len(Injuries) - cnt)/2
            totalLength = int(totalLength)
        
            #print("totalLength")
            #print(totalLength)
        
            for j in range(0, totalLength):
                injuryList.append(Injuries[j*2].text)
                gameMissedList.append(Injuries[j*2+1].text)
                seasonList.append(Others[j*3].text)
                fromList.append(Others[j*3+1].text)
                toList.append(Others[j*3+2].text)
                dayMissedList.append(Days[j*2].text)
   

            injuryCntList.append(totalLength);
            driver.get(init_url)
        
        for i in range(0,playerNum):
            for j in range(0, injuryCntList[i]):
                playerAddList.append(playerList[i])
                leagueList[l] = leagueList[l] + 1
        print("next Team")
        driver.get(leagueUrl)
        
    driver.get(url)




#league Add list 만들기 league Country Add list 만들기
leagueAddList = []

for i in range(0, leagueList[0]):
    leagueAddList.append('Premier League')
for i in range(0, leagueList[1]):
    leagueAddList.append('Serie A')
for i in range(0, leagueList[2]):
    leagueAddList.append('Laliga')
for i in range(0, leagueList[3]):
    leagueAddList.append('Bundesliga')
for i in range(0, leagueList[4]):
    leagueAddList.append('Ligue 1')
for i in range(0, leagueList[5]):
    leagueAddList.append('Liga NOS')
for i in range(0, leagueList[6]):
    leagueAddList.append('Eredivise')
    
#dataframe 만들기 
df = pd.DataFrame({"season": seasonList, "injury": injuryList, "from": fromList, "to": toList, "day_missed": dayMissedList, "game_missed": gameMissedList})
df.insert(0, 'player', playerAddList)
df.insert(1, 'league', leagueAddList)

df

playerNum
25
Player
Ederson
Player
Zack Steffen
Player
Rúben Dias
Player
Aymeric Laporte
Player
Nathan Aké
Player
John Stones
Player
Eric García
Player
Philippe Sandler
Player
Oleksandr Zinchenko
Player
Benjamin Mendy
Player
João Cancelo
Player
Kyle Walker
Player
Rodri
Player
Fernandinho
Player
Ilkay Gündogan
Player
Kevin De Bruyne
Player
Bernardo Silva
Player
Raheem Sterling
Player
Phil Foden
Player
Jack Harrison
Player
Ferran Torres
Player
Riyad Mahrez
Player
Patrick Roberts
Player
Gabriel Jesus
Player
Sergio Agüero
next Team
playerNum
31
Player
Alisson
Player
Caoimhin Kelleher
Player
Adrián
Player
Virgil van Dijk
Player
Joe Gomez
Player
Ozan Kabak
Player
Joel Matip
Player
Nathaniel Phillips
Player
Rhys Williams
Player
Ben Davies
Player
Andrew Robertson
Player
Konstantinos Tsimikas
Player
Trent Alexander-Arnold
Player
Neco Williams
Player
Fabinho
Player
Naby Keïta
Player
Georginio Wijnaldum
Player
Thiago
Player
Curtis Jones
Player
Jordan Henderson
Player
Alex Oxlade-Chamberlain
Playe

,player,league,season,injury,from,to,day_missed,game_missed
0,Ederson,Premier League,20/21,Virus Infection,"Dec 27, 2020","Jan 12, 2021",16 days,3
1,Ederson,Premier League,19/20,Ill,"Dec 30, 2019","Jan 7, 2020",8 days,3
2,Ederson,Premier League,19/20,Ill,"Dec 8, 2019","Dec 14, 2019",6 days,1
3,Ederson,Premier League,19/20,Muscular problems,"Nov 6, 2019","Nov 22, 2019",16 days,2
4,Ederson,Premier League,17/18,Facial Injury,"Sep 10, 2017","Sep 12, 2017",2 days,-
...,...,...,...,...,...,...,...,...
18163,Vitalie Damascan,Eredivise,19/20,Ankle Injury,"Sep 9, 2019","Oct 7, 2019",28 days,5
18164,Vitalie Damascan,Eredivise,19/20,Ankle problems,"Sep 9, 2019","Oct 7, 2019",28 days,5
18165,Finn Stokkers,Eredivise,16/17,Ankle Injury,"Jul 15, 2016","Aug 5, 2016",21 days,-
18166,Finn Stokkers,Eredivise,15/16,Ankle Injury,"Oct 18, 2015","Nov 10, 2015",23 days,5


In [2]:
df.to_csv('C:\\Users\\ekkic\\Desktop\\data science 눈문(Junbum Kim)(2021)\\injury_data.csv')